In [ ]:
import numpy as np

import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import pearsonr, spearmanr
from statsmodels.stats.multitest import multipletests
import statsmodels.api as sm

from pathlib import Path

DATA_PATH = '../../data'
OUT_PATH = '../../out'
Path(OUT_PATH).mkdir(parents=True, exist_ok=True)

# Data loading

Loading participant master data file, linking food record data, and filtering down to records with complete information.

## Participants master

In [ ]:
participants_master = pd.read_excel(
    DATA_PATH+'/main/participants_master.xlsx',
    index_col='id'
)

## Food intake records

In [ ]:
intakes_per_person = pd.read_excel(
    DATA_PATH+'/main/intakes_detailed_with_asep.xlsx',
    index_col='id'
)

## Bloodwork

In [ ]:
huslab = pd.read_excel(
    DATA_PATH+'/main/huslab_results.xlsx',
)

## Merging

In [ ]:
master_data = participants_master.merge(
    intakes_per_person,
    left_index=True,
    right_index=True,    
).merge(
    huslab,
    left_index=True,
    right_on='id',
    how='left',
)

## Filtering for valid data points

In [ ]:
fltr = (master_data.diet_group.notnull()) & (master_data.fr_days>=3)
participants = master_data[fltr]

children = participants[participants.age_q < 3650] #choosing 10y as cutoff as study is of preschool children

# Analysis

First we calculate Pearson correlations for macro intake groups with ASEP, and apply a Benjamini Hochberg correction. The results are presented in a table written out to a file.

In [ ]:
#reused in several plots
diet_classes=['vegan','vegetarian','mixed diet']

In [ ]:
def asep_correlation_table(cols, df=children):
    df = df[cols+['asep']].dropna()
    n = len(df.index)

    table = []
    for c in df.columns[:-1]: #all but last column, i.e. asep
        pr, pp = pearsonr(df.asep,df[c])
        sr, sp = spearmanr(df.asep,df[c])
        table.append([n,'asep',c,pr,pp,sr,sp])

    out = pd.DataFrame(table,columns=['n','x','y','pearson_r','pearson_p','spearman_r','spearman_p'])
    out['pearson_fdr_bh_0_05'] = multipletests(out['pearson_p'], alpha=0.05, method='fdr_bh')[0]
    out['spearman_fdr_bh_0_05'] = multipletests(out['spearman_p'], alpha=0.05, method='fdr_bh')[0]
    out = out.sort_values(by='pearson_p')

    return out

In [ ]:
df = asep_correlation_table(['CHOLE_mg_per_MJ','FIBC_g_per_MJ','ep-FAPU','e-FASAT',])
df.to_excel(OUT_PATH+'/correlation_macro_intakes_children.xlsx')

In [ ]:
df = children.groupby('diet_group').asep.describe()
display(df)
df.to_excel(OUT_PATH+'/asep_describe_by_diet_children.xlsx')

# Figures

In [ ]:
scale=10
sns.set_theme(style='white',font_scale=2)
sns.set_context("paper")

tricolor_palette=['#55a868','#dd8452','#4c72b0']

In [ ]:
from string import ascii_lowercase

markers=['o','s','^']

def regplot_asep(df,ax,y,y_label,set_xlabel=True):
    ax.set_xlim(-0.01, df.asep.max()+0.01)
    sns.regplot(ax=ax, x='asep', y=y, scatter=False, data=df)
    sns.scatterplot(
        ax=ax, 
        x='asep', 
        y=y, 
        hue='diet_group',
        hue_order=diet_classes,
        style='diet_group',
        markers=markers,
        data=df, 
        s=30*scale,
        palette=tricolor_palette,
        legend=False
    )
    if set_xlabel:
        ax.set_xlabel('Animal source energy proportion')
    else:
        ax.set_xlabel(None)
    ax.set_ylabel(y_label)


from matplotlib.lines import Line2D
def splats(n):
    return [
        Line2D(
            [0], 
            [0], 
            marker=markers[i],
            color='w',
            markerfacecolor=tricolor_palette[i],
            markersize=15
        )
        for i in range(0,n)
    ]


def fig_of_regplots(df,ys,cols=2,legend=True):

    plt.close()
    sns.set_theme(style='white',font_scale=2)
    
    rows = int(len(ys)/cols)+(len(ys)%cols > 0)
    h = rows*scale
    if rows > 1: h+=2
    w = cols*scale+2

    fig, axs = plt.subplots(rows,cols,figsize=(w,h))
        
    if rows > 1:
        axs_flat = [ax for row in axs for ax in row]
    else:
        axs_flat = axs

    l = list(zip(ys.keys(),ys.values(),axs_flat))
    i = 0
    for t in l:
        ax = t[2]
        y = t[0]
        regplot_asep(df,ax,y,t[1],set_xlabel=True)
        i+=1

    for ax in axs_flat[len(ys):]:
        fig.delaxes(ax)
        
    from matplotlib.lines import Line2D
    
    if legend:
        plt.figlegend(splats(3),diet_classes,loc='right')
        
    return fig, axs

## Intakes

Drawing scatterplots with linear regresssion for intakes and ASEP

In [ ]:
# a multi-panel graph of macro intakes linear model correlation with asep
sns.set_context("paper")

ys = {
    'ep-FAT': 'Total Fat (E%)',
    'ep-FAPU': 'Polyunsaturated fatty acids (E%)',
    'ep-FASAT': 'Saturated fatty acids (E%)',
    'ep-CHOAVL': 'Carbohydrates (E%)',
    'ep-PROT': 'Protein (E%)',
    'FIBC_g_per_MJ': 'Fiber (g/MJ)',
}

fig, axs = fig_of_regplots(children,ys,cols=3)

#plt.show()
plt.savefig(OUT_PATH+'/macro_intakes_children.png')
plt.close()

In [ ]:
# a multi-panel graph of fat metabolism intakes linear model correlation with asep
sns.set_context("paper")

ys = {
    'ep-FASAT': 'Saturated fatty acids (E%)',
    'ep-FAPU': 'Polyunsaturated fatty acids (E%)',
    'CHOLE_mg_per_MJ': 'Cholesterol (mg/MJ)',
    'FIBC_g_per_MJ': 'Fiber (g/MJ)',    
}

fig, axs = fig_of_regplots(children,ys,cols=2)

#plt.show()
plt.savefig(OUT_PATH+'/fat_fibc_intakes_children.png')
plt.close()

In [ ]:
# a multi-panel graph of micronutrient intakes linear model correlation with asep
sns.set_context("paper")

ys = {
    'F20D5N3_daily_mean': 'EPA (mg/d)',
    'F22D6N3_daily_mean': 'DHA (mg/d)',
    'F18D2CN6_daily_mean': 'Linoleic acid (mg/d)',
    'F18D3N3_daily_mean': 'Alpha-linoleic acid (mg/d)',
    'FE_daily_mean': 'Iron (mg/d)',
    'CA_daily_mean': 'Calcium (mg/d)',
    'FOL_daily_mean': 'Folate (HPLC) (μg/d)',
    'VITC_daily_mean': 'Ascorbic acid (mg/d)',
}

fig, axs = fig_of_regplots(children,ys,cols=3)

#plt.show()
plt.savefig(OUT_PATH+'/micro_intakes_children.png')
plt.close()

In [ ]:
#histogram of asep values in buckets of 5% and coloured according to reported diet

fg = sns.displot(
    children,
    x='asep',
    binwidth=0.05,
    hue='diet_group',
    hue_order=diet_classes,
    multiple='stack',
    height=scale,
    palette=tricolor_palette
)
fg.axes[0,0].set_xlabel('Animal source energy proportion')
fg.axes[0,0].set_ylabel('Number of participants')

plt.savefig(OUT_PATH+'/displot_asep_diet_class_children.png')
plt.close()

# Biomarkers

Drawing a grid of regplots and producing a table of correlation values for groups of biomarkers:
- lipid
- iron

In [ ]:
children_valid_labwork = children[
    (children.blood_time_of_day < '11:00.00')
    &
    children.id != 'M3384' #reported to have eaten in the morning before blood sample
]

In [ ]:
lipid_biomarkers = {
    'fP.Kol.HDL': 'HDL Cholesterol (mmol/l)',
    'fP.Kol.LDL': 'LDL Cholesterol (mmol/l)',
    'fP.Kol': 'Cholesterol (mmol/l)',
    'fP.Trigly': 'Triglycerides (mmol/l)',
    'fP.LipoA1': 'Lipoprotein A1 (g/l)',
    'fP.LipoB': 'Lipoprotein B (g/l)',
    'fP.ApoBperA1': 'apoB/apoA1 ratio',
}

sns.set_context("paper")
fig, axs = fig_of_regplots(
    children_valid_labwork,
    lipid_biomarkers,
    cols=3
)
plt.savefig(OUT_PATH+'/lipid_biomarkers_children.png')
plt.close()

df = asep_correlation_table(
    list(lipid_biomarkers.keys()),
    df=children_valid_labwork
)
df.to_excel(OUT_PATH+'/correlation_lipid_biomarkers_children.xlsx')

In [ ]:
iron_biomarkers = {
    'B.Hb': 'Hemoglobin (g/l)',
    'B.HKR': 'Hematocrit (%)',
    'E.MCV': 'Mean corpuscular volume (fl)',
    'E.RDW': 'Red cell distribution width (%)',
    'E.MCH': 'Mean corpuscular hemoglobin (pg)',
    'E.MCHC': 'Mean corpuscular hemoglobin concentration (g/l)',
}


sns.set_context("paper")
fig, axs = fig_of_regplots(
    children_valid_labwork,
    iron_biomarkers,
    cols=3
)
plt.savefig(OUT_PATH+'/iron_biomarkers_children.png')
plt.close()

df = asep_correlation_table(list(iron_biomarkers.keys()))
df.to_excel(OUT_PATH+'/correlation_iron_biomarkers_children.xlsx')

# ICDAM Poster

Graphs for conference poster

In [ ]:
# a multi-panel graph of macro intakes linear model correlation with asep
sns.set_context("poster")

ys = {
    'ep-FAT': 'Total Fat (E%)',
    'ep-FAPU': 'Polyunsaturated fatty acids (E%)',
    'ep-FASAT': 'Saturated fatty acids (E%)',
#    'ep-CHOAVL': 'Carbohydrates (E%)',
#    'ep-PROT': 'Protein (E%)',
    'FIBC_g_per_MJ': 'Fiber (g/MJ)',
}

fig, axs = fig_of_regplots(children,ys,cols=2, legend=False)

plt.savefig(OUT_PATH+'/macro_intakes_children_poster.png')
#plt.show()
plt.close()

In [ ]:
ys = {
    'fP.Kol.LDL': 'LDL Cholesterol (mmol/l)',
    'fP.Trigly': 'Triglycerides (mmol/l)',
}

sns.set_context("poster")
fig, axs = fig_of_regplots(
    children_valid_labwork,
    ys,
    cols=2,
    legend=False
)

plt.savefig(OUT_PATH+'/lipid_biomarkers_children_poster.png')
plt.close()

In [ ]:
#histogram of asep values in buckets of 5% and coloured according to reported diet
sns.set_context("poster")

fg = sns.displot(
    children,
    x='asep',
    binwidth=0.05,
    hue='diet_group',
    hue_order=diet_classes,
    multiple='stack',
    height=scale,
    palette=tricolor_palette
)
fg.axes[0,0].set_xlabel('Animal source energy proportion')
fg.axes[0,0].set_ylabel('Number of participants')

sns.move_legend(fg, 'upper right',bbox_to_anchor=(0.78, 0.9))

plt.savefig(OUT_PATH+'/displot_asep_diet_class_children_poster.png')
#plt.show()
plt.close()